<p style="font-size:20px;">Simulator: QiskitAer, FakeTorino, ideal device noise simulator (no shot noise). H4 (rectangular) in minimal basis (STO-3G), JW mapping, 3-layered tUPS. With 40 samples generated randomly, number of parameters is 27, number of non Clifford parameters k = 3. Fitting methods: Square regression with a linear ansatz, Weighted Least Squares, Nonlinear Fitting, XGB, MLP.</p>

In [16]:
import pyscf
import slowquant.SlowQuant as sq
from qiskit_nature.second_q.mappers import JordanWignerMapper
from slowquant.qiskit_interface.interface import QuantumInterface
from slowquant.qiskit_interface.wavefunction import WaveFunction
#from slowquant.qiskit_interface.linear_response.projected import quantumLR

from qiskit_ibm_runtime.fake_provider import FakeTorino

from qiskit.primitives import Estimator

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_aer.primitives import Sampler
from qiskit_aer.primitives import SamplerV2

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import curve_fit
import random

from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

In [2]:
mol = pyscf.M(atom="""H 0.0 0.0 0.0;
                      H 1.5 0.0 0.0;
                      H 0.0 1.8 0.0;
                      H 1.5 1.8 0.0;""", basis="sto-3g", unit="angstrom")
rhf = pyscf.scf.RHF(mol).run()

sampler = Estimator()
primitive = sampler
mapper = JordanWignerMapper()
# For H4 you can make the wavefunction better by increasing n_layers.
# n_layers: 3 will prob. give almost the FCI solution.
QI = QuantumInterface(primitive, "tUPS", mapper, ansatz_options={"n_layers": 3, "do_pp": True}, ISA = True)

WF = WaveFunction(
    mol.nao * 2,
    mol.nelectron,
    (4, 4),
    rhf.mo_coeff,
    mol.intor("int1e_kin") + mol.intor("int1e_nuc"),
    mol.intor("int2e"),
    QI,
    #include_active_kappa = True
)

WF.run_vqe_2step("RotoSolve", maxiter=3)
WF.run_vqe_2step("SLSQP", maxiter=40)

#no noise
nonclif_ground_state_energy = WF.energy_elec
print("Non-Clifford Ground state energy:", nonclif_ground_state_energy)

#device noise
# Update the primitive with simulated noise
backend = FakeTorino()
QI.pass_manager = generate_preset_pass_manager(3,backend=backend,seed_transpiler=123) # seeded standard transpiler
QI.do_postselection = False
QI.do_M_mitigation = False
noise_model = NoiseModel.from_backend(backend)
sampler = Sampler(backend_options={"noise_model":noise_model})
WF.change_primitive(sampler)    
# Calculate the ground state energy using the noisy simulator
noisy_nonclif_ground_state_energy = WF.energy_elec
print("Noisy Non-Clifford Ground state energy:",noisy_nonclif_ground_state_energy)

converged SCF energy = -1.77674731624872
Number of shots is None. Ideal simulator is assumed.
You selected ISA but did not pass a PassManager. Standard internal transpilation will use backend None with optimization level 3
Full optimization
Iteration # | Iteration time [s] | Electronic energy [Hartree]
--------Ansatz optimization
--------Iteration # | Iteration time [s] | Electronic energy [Hartree]


/tmp/ipykernel_591019/4086987143.py:7: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  sampler = Estimator()


--------     1      |        50.82       |     -3.5535136105572458    
--------     2      |        51.02       |     -3.5832852103235662    
--------     3      |        52.05       |     -3.5860285285231805    
Full optimization
Iteration # | Iteration time [s] | Electronic energy [Hartree]
--------Ansatz optimization
--------Iteration # | Iteration time [s] | Electronic energy [Hartree]
--------     1      |        20.49       |     -3.5863422696342351    
--------     2      |        20.34       |     -3.5866118237282123    
--------     3      |        20.36       |     -3.5867386834939445    
--------     4      |        20.17       |     -3.5872715356047364    
--------     5      |        20.31       |     -3.5884052688525241    
--------     6      |        20.27       |     -3.0422668988251416    
--------     7      |        20.47       |     -3.5740847223601957    
--------     8      |        20.35       |     -3.5889900038251907    
--------     9      |        20.31     

In [4]:
#prepare training set
clif_ground_state_energies = []
noisy_clif_ground_state_energies = []
n_replacements = 3

for n in range(40):
    QI = QuantumInterface(primitive, "tUPS", mapper, ansatz_options={"n_layers": 3, "do_pp": True}, ISA = True)

    WF = WaveFunction(
    mol.nao * 2,
    mol.nelectron,
    (4, 4),
    rhf.mo_coeff,
    mol.intor("int1e_kin") + mol.intor("int1e_nuc"),
    mol.intor("int2e"),
    QI,
    #include_active_kappa = True
    )

    indices_to_replace = random.sample(range(len(WF.ansatz_parameters)), n_replacements)
    for index in indices_to_replace:
        WF.ansatz_parameters[index] = random.uniform(0, 2*np.pi)
    print(WF.ansatz_parameters)

    clif_ground_state_energy = WF.energy_elec
    clif_ground_state_energies.append(clif_ground_state_energy)
    
    backend = FakeTorino()
    QI.pass_manager = generate_preset_pass_manager(3,backend=backend,seed_transpiler=123) # seeded standard transpiler
    QI.do_postselection = False
    QI.do_M_mitigation = False
    noise_model = NoiseModel.from_backend(backend)
    sampler = Sampler(backend_options={"noise_model":noise_model})
    WF.change_primitive(sampler)   
    noisy_clif_ground_state_energy = WF.energy_elec
    noisy_clif_ground_state_energies.append(noisy_clif_ground_state_energy)

Number of shots is None. Ideal simulator is assumed.
You selected ISA but did not pass a PassManager. Standard internal transpilation will use backend None with optimization level 3
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.15785647893655746, 0.0, 0.0, 4.0306986917429395, 5.776398626048327, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Using this function is only recommended for switching from ideal simulator to shot-noise or quantum hardware.
                 Multiple switching back and forth can lead to un-expected outcomes and is an experimental feature.

Reset RDMs, energies, and QI metrics.
Number of shots is None. Ideal simulator is assumed.
Your settings are:
 Ansatz:              tUPS
 Number of shots:     None
 ISA                  True
 Transpiled circuit   True
 Primitive:           Sampler
 Post-processing:     False
 Circuit layout:      [17, 25, 35, 24, 28, 26, 27, 44]
 Non-local gates:     405
 Transpilation strategy: External PassManager
 Primit

In [8]:
print(clif_ground_state_energies)
print(noisy_clif_ground_state_energies)

[np.float64(-3.2633339089131095), np.float64(-3.3652471970891846), np.float64(-2.8936285852412955), np.float64(-3.289295016220068), np.float64(-2.880049401172856), np.float64(-3.2795660830226994), np.float64(-3.0648854647313777), np.float64(-3.3407538571388393), np.float64(-3.5310383768212183), np.float64(-2.6205310448932657), np.float64(-3.2982858430276956), np.float64(-3.20352831393017), np.float64(-2.981463077638647), np.float64(-3.370973161674473), np.float64(-2.8423390239012747), np.float64(-2.760037615967007), np.float64(-3.104132502517762), np.float64(-3.0371723474629233), np.float64(-3.269450699857538), np.float64(-2.6546184034990614), np.float64(-3.394440650673899), np.float64(-2.742236722182637), np.float64(-2.9508428793804686), np.float64(-3.085532750763045), np.float64(-3.382930829883438), np.float64(-2.542281724905676), np.float64(-2.5651775940237402), np.float64(-2.9754251839481336), np.float64(-2.827936941018348), np.float64(-2.52295672408021), np.float64(-3.079620815456

In [5]:
#data preparation
nonclif_ground_state_energies = [nonclif_ground_state_energy for _ in range(len(clif_ground_state_energies))]
noisy_nonclif_ground_state_energies = [noisy_nonclif_ground_state_energy for _ in range(len(noisy_clif_ground_state_energies))]
X = np.array(noisy_clif_ground_state_energies).reshape(-1, 1)
y = np.array(clif_ground_state_energies)
X_test = np.array(noisy_nonclif_ground_state_energies).reshape(-1, 1)
y_test = np.array(nonclif_ground_state_energies)

In [20]:
#square regression with a linear ansatz
def cost_function(a, X_exact, X_noisy):
    a1, a2 = a  # 将参数 a 拆分为 a1 和 a2
    # 计算代价函数 C 的值
    C = np.sum((X_exact - (a1 * X_noisy + a2))**2)
    return C
X_exact = np.array(clif_ground_state_energies)
X_noisy = np.array(noisy_clif_ground_state_energies)
# 初始猜测值 a1 和 a2
initial_guess = [1, 0]
# 最小化代价函数
result = minimize(cost_function, initial_guess, args=(X_exact, X_noisy))
# 得到最优的 a1 和 a2
a1_optimal, a2_optimal = result.x
#print(f"最优的 a1 值: {a1_optimal}")
#print(f"最优的 a2 值: {a2_optimal}")
y_pred_reg = a1_optimal*noisy_nonclif_ground_state_energy + a2_optimal
print('result of square regression:', y_pred_reg)

result of square regression: -3.232249426618649


In [19]:
# Weighted Least Squares
def weighted_cost_function(a, X_exact, X_noisy, weights):
    a1, a2 = a  # 拆分参数
    residuals = X_exact - (a1 * X_noisy + a2)
    # 加权平方和
    C = np.sum(weights * residuals**2)
    return C

# 输入数据
X_exact = np.array(clif_ground_state_energies)
X_noisy = np.array(noisy_clif_ground_state_energies)

# 假设权重为噪声的不确定性 (例如误差) 的倒数平方
# 示例权重：可以根据具体问题替换为真实的不确定性
noise_uncertainties = np.random.rand(len(X_noisy)) * 0.1  # 示例噪声
weights = 1 / (noise_uncertainties**2)

# 初始猜测值
initial_guess = [1, 0]

# 使用 scipy.optimize.minimize 进行优化
result = minimize(weighted_cost_function, initial_guess, args=(X_exact, X_noisy, weights))

# 提取最优参数
a1_optimal, a2_optimal = result.x

# 打印结果
#print(f"最优的 a1 值: {a1_optimal}")
#print(f"最优的 a2 值: {a2_optimal}")

# 对新的数据进行预测
y_pred_reg = a1_optimal * noisy_nonclif_ground_state_energy + a2_optimal
print('Result of Weighted Least Squares:', y_pred_reg)

Result of Weighted Least Squares: -3.2480645597035807


In [21]:
#Nonlinear Fitting
# 定义非线性拟合函数
def nonlinear_model(X_noisy, a1, a2, a3):
    return a1 * X_noisy**2 + a2 * X_noisy + a3

# 输入数据
X_exact = np.array(clif_ground_state_energies)
X_noisy = np.array(noisy_clif_ground_state_energies)

# 使用 curve_fit 进行非线性拟合
# 初始猜测值 [a1, a2, a3]
initial_guess = [1, 1, 0]
params, covariance = curve_fit(nonlinear_model, X_noisy, X_exact, p0=initial_guess)

# 提取拟合参数
a1_optimal, a2_optimal, a3_optimal = params
#print(f"最优的 a1 值: {a1_optimal}")
#print(f"最优的 a2 值: {a2_optimal}")
#print(f"最优的 a3 值: {a3_optimal}")

# 对新的数据进行预测
y_pred_reg = nonlinear_model(noisy_nonclif_ground_state_energy, a1_optimal, a2_optimal, a3_optimal)
print('Result of Nonlinear Fitting:', y_pred_reg)

Result of Nonlinear Fitting: -3.230130282200639


In [12]:
#XGBoost
xgb_model = XGBRegressor(max_depth=20, eta=0.0005, subsample=0.7, colsample_bytree=0.7, objective='reg:absoluteerror', n_estimators=1000, random_state=42)
xgb_model.fit(X, y)
y_pred_xgb = xgb_model.predict(X_test)
# 计算均方误差
#mse_corrected_XGB = mean_squared_error(y_test, y_pred_xgb)
#print('mse:', mse_corrected_XGB)
print('result of xgb:', y_pred_xgb[0])

result of xgb: -3.1624904


In [14]:
#MLP
# 归一化数据
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# 训练MLP模型
mlp_model = MLPRegressor(hidden_layer_sizes=(120, 60), activation='relu', solver='adam', max_iter=2000, alpha=0.001, random_state=42)
mlp_model.fit(X_scaled, y)
X_test_scaled = scaler.transform(X_test)
y_pred_mlp = mlp_model.predict(X_test_scaled)
#mse_corrected_MLP = mean_squared_error(y_test, y_pred_mlp)
#print('mse:', mse_corrected_MLP)
print('result of mlp:', y_pred_mlp[0])

result of mlp: -3.222777595510765
